In [ ]:
## Movie Recommendtation Engine

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, PowerTransformer, OrdinalEncoder, LabelEncoder, FunctionTransformer, PolynomialFeatures
from sklearn.linear_model import SGDRegressor, Ridge, Lasso, ElasticNet
from sklearn.utils import shuffle
from sklearn.preprocessing import PolynomialFeatures, KBinsDiscretizer
from sklearn.tree import DecisionTreeRegressor
import scipy.stats as stats
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, KFold
from dateutil.parser import parse
from sklearn.feature_extraction import FeatureHasher
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
import ast
from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem import PorterStemmer
import nltk

In [2]:
# nltk.download('all')

### Importing Data set Movie and Movie credit

In [3]:
# Load the movie dataset 
df = pd.read_csv('movie.csv', low_memory=False)

In [4]:
# This dataframe contain movie credits (cast and crew details)
df_2 = pd.read_csv('Movie_credits.csv', low_memory=False)

In [5]:
df.shape

(4803, 20)

In [6]:
df_2.shape

(4803, 4)

In [7]:
# Display the first few rows of the newly created DataFrame to verify its structure
# df.head(2)

In [8]:
# Display the first few rows of the newly created DataFrame to verify its structure
# df_2.head()

### Merging both dataset based on ID

In [9]:
# Merge two dataframes to consolidate movie data with additional cast and crew information
new_df = df.merge(df_2[['movie_id', 'cast', 'crew']],
                   left_on='id',
                   right_on='movie_id',
                   how='left')

In [10]:
new_df.shape

(4803, 23)

In [11]:
# Remove unnecessary columns from the merged DataFrame to simplify and focus on relevant data
filtered_df = new_df.drop(['movie_id', 'homepage', 'production_countries', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'vote_average', 'vote_count',  'original_title'], axis=1)

In [12]:
filtered_df.head(3)

,budget,genres,id,keywords,original_language,overview,popularity,production_companies,spoken_languages,title,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [13]:
# null_counts = filtered_df.isnull().sum()
# print(null_counts)

In [14]:
# dup_rows = filtered_df.duplicated().sum()
# print(f"dup rows: {dup_rows}")

In [15]:
# Display the datatype of each columns in dataframe
print(filtered_df.dtypes)

budget                    int64
genres                   object
id                        int64
keywords                 object
original_language        object
overview                 object
popularity              float64
production_companies     object
spoken_languages         object
title                    object
cast                     object
crew                     object
dtype: object


In [16]:
print(filtered_df['genres'][1])

[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]


In [17]:
#Converting  Genres to genre_names  In normal format 
filtered_df['genre_names'] = filtered_df['genres'].apply(lambda x: ', '.join([genre['name'] for genre in eval(x)]) if isinstance(x, str) else ', '.join([genre['name'] for genre in x]))

In [18]:
# Same Porcess Applied on keywords to keywords_name 
filtered_df['keywords_name'] = filtered_df['keywords'].apply(lambda x: ', '.join([keywords['name'] for keywords in eval(x)]) if isinstance(x, str) else ', '.join([keywords['name'] for keywords in x]))

In [19]:
# Drop additional columns from the filtered dataframe that are not required further
extracted_df = filtered_df.drop(['genres', 'keywords', 'spoken_languages', 'original_language'], axis=1)

In [20]:
# Display the first three rows
extracted_df.head(3)

,budget,id,overview,popularity,production_companies,title,cast,crew,genre_names,keywords_name
0,237000000,19995,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony..."
1,300000000,285,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t..."
2,245000000,206647,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6..."


In [21]:
# Create a new DataFrame with selected columns to organize and re-arrange the columns
new_df = extracted_df[['id', 'title', 'budget', 'overview', 'popularity', 'genre_names', 'keywords_name', 'cast', 'crew',  'production_companies',]].copy()

In [22]:
# Display the first three rows data
new_df.head(3)

,id,title,budget,overview,popularity,genre_names,keywords_name,cast,crew,production_companies
0,19995,Avatar,237000000,"In the 22nd century, a paraplegic Marine is di...",150.437577,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,285,Pirates of the Caribbean: At World's End,300000000,"Captain Barbossa, long believed to be dead, ha...",139.082615,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,206647,Spectre,245000000,A cryptic message from Bond’s past sends him o...,107.376788,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."


In [23]:
# Convert string representations of lists in the 'crew' column back to Python list objects

new_df['crew'] = new_df['crew'].apply(ast.literal_eval)

# new_df['director_name'] = new_df['crew'].apply(
#     lambda x: next((crew['name'] for crew in x if crew['job'].lower() == 'director'), None)
# )

# Extract the name of the director from the 'crew' data, remove spaces from names for consistency, and add it to a new 'director_name' column
new_df['director_name'] = new_df['crew'].apply(
   lambda x: next((crew['name'].replace(' ', '') for crew in x if crew['job'].lower() == 'director'), None)
)

In [24]:
# Display the first three rows data
new_df.head(3)

,id,title,budget,overview,popularity,genre_names,keywords_name,cast,crew,production_companies,director_name
0,19995,Avatar,237000000,"In the 22nd century, a paraplegic Marine is di...",150.437577,"Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{'credit_id': '52fe48009251416c750aca23', 'de...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...",JamesCameron
1,285,Pirates of the Caribbean: At World's End,300000000,"Captain Barbossa, long believed to be dead, ha...",139.082615,"Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",GoreVerbinski
2,206647,Spectre,245000000,A cryptic message from Bond’s past sends him o...,107.376788,"Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",SamMendes


In [25]:
# Combine relevant information into a single column for consolidated data analysis
new_df['merged_info'] = new_df['overview'] + ' ' + new_df['genre_names'] + ' ' +  new_df['keywords_name'] + ' ' + new_df['director_name']

In [26]:
# Drop columns that are no longer needed after merging their information into the 'merged_info' column
new_df = new_df.drop(['budget', 'overview', 'popularity', 'genre_names', 'keywords_name', 'cast', 'crew', 'production_companies', 'director_name'], axis=1)

In [27]:
new_df.head(3)

,id,title,merged_info
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...


In [28]:
print(new_df['merged_info'][1])

Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems. Adventure, Fantasy, Action ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, shipwreck, strong woman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercreditsstinger GoreVerbinski


In [29]:
# Remove rows with missing information in the 'merged_info' column to clean up the dataset
new_df = new_df.dropna(subset=['merged_info'])

In [30]:
# Calculate the count of null values in each column
null_counts = new_df.isnull().sum()
print(null_counts)

id             0
title          0
merged_info    0
dtype: int64


Stemming

In [31]:
# Instantiate a PorterStemmer object - Stemming
ps = PorterStemmer()

# Apply stemming to each word in the 'merged_info' column to reduce words to their root forms
new_df['merged_info'] = new_df['merged_info'].apply(
    lambda x: ' '.join([ps.stem(word) for word in nltk.word_tokenize(str(x))])
)

Bag of Words

In [43]:
# Using sklearn library load their funciton CountVectorizer and create the vectors through it from the column `merged_info` - Bag of words
vectorizer = CountVectorizer(
    stop_words='english',
    # max_features=4000,
    lowercase=True,
)
bow_matrix = vectorizer.fit_transform(new_df['merged_info'])
feature_names = vectorizer.get_feature_names_out()

In [33]:
bow_matrix.shape

(4770, 19312)

In [34]:
print(len(feature_names))

19312


In [35]:
print(feature_names[:500])

['00' '000' '007' '07am' '10' '100' '1000' '101' '108' '10th' '11' '114'
 '117' '118' '119' '11th' '12' '1200' '1215' '1250' '125th' '12th' '13'
 '1300' '13th' '14' '140' '1408' '142' '1429' '148' '14pm' '14th' '15'
 '150' '150th' '1520' '1536' '15th' '16' '1600' '161' '1630' '1644' '1681'
 '1691' '16th' '17' '170' '1700' '173rd' '1748' '1776' '17th' '18' '180'
 '1800' '1818' '1820' '1824' '1831' '1834' '1836' '1838' '1839' '1841'
 '1845' '1850' '1856' '1857' '1860' '1862' '1863' '1870' '1875' '1876'
 '1879' '1880' '1882' '1885' '1889' '1890' '18th' '19' '1900' '1903'
 '1905' '191' '1910' '1911' '1912' '1914' '1915' '1917' '1918' '1919'
 '1920' '1921' '1922' '1924' '1925' '1926' '1927' '1928' '1929' '1930'
 '1931' '1932' '1933' '1934' '1936' '1937' '1939' '1940' '1941' '1942'
 '1943' '1944' '1945' '1946' '1947' '1949' '1950' '1951' '1952' '1953'
 '1954' '1955' '1956' '1957' '1958' '1959' '1960' '1962' '1964' '1965'
 '1966' '1967' '1968' '1969' '1970' '1971' '1972' '1973' '1974' '1975'


In [36]:
# Calculate the cosine similarity between the vectors in the bag-of-words matrix
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(bow_matrix, bow_matrix)

In [37]:
cosine_sim

array([[1.        , 0.0693167 , 0.04982042, ..., 0.06947125, 0.0158193 ,
        0.        ],
       [0.0693167 , 1.        , 0.04259177, ..., 0.05345225, 0.        ,
        0.01974496],
       [0.04982042, 0.04259177, 1.        , ..., 0.03414939, 0.        ,
        0.        ],
       ...,
       [0.06947125, 0.05345225, 0.03414939, ..., 1.        , 0.048795  ,
        0.03166238],
       [0.0158193 , 0.        , 0.        , ..., 0.048795  , 1.        ,
        0.07209841],
       [0.        , 0.01974496, 0.        , ..., 0.03166238, 0.07209841,
        1.        ]])

In [38]:
cosine_sim[0].shape

(4770,)

In [39]:
cosine_sim[0]

array([1.        , 0.0693167 , 0.04982042, ..., 0.06947125, 0.0158193 ,
       0.        ])

In [40]:
# Define a function to recommend movies based on their cosine similarity and also added the in
def recommend(movie):
    # Retrieve the index/id of the specified movie in the dataframe to use in similarity comparisons  
    index = new_df[new_df['title'] == movie].index[0]
    # Sort the cosine similarity scores for the specified movie in descending order to identify the most similar movies
    distances = sorted(list(enumerate(cosine_sim[index])),reverse=True,key = lambda x: x[1])
    # Print the titles of the top 5 movies that are most similar to the specified movie   
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [42]:
# Use the recommend function to find movies similar to keyword
recommend("Spider-Man")

Spider-Man 3
The Amazing Spider-Man 2
Spider-Man 2
Superman III
The Amazing Spider-Man
